In [ ]:
import re
import os
from collections import defaultdict
import regex as re
import numpy as np
from collections import defaultdict, Counter, OrderedDict, namedtuple

from torchtext.vocab import build_vocab_from_iterator, vocab
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import chain

import nltk
import sklearn
import scipy.stats
# from sklearn.metrics import make_scorer
# from sklearn.cross_validation import cross_val_score
# from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


import pandas as pd
import os
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import pycrfsuite
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents



In [ ]:
def accent_normalize(string):
    for char in accents_dict:
        string = re.sub(char, accents_dict[char],string)
    return string

normalizer = normalizers.Sequence([NFD(),StripAccents()])

In [ ]:
normalize = lambda x: re.sub("'","",normalizer.normalize_str(x))

# START = "<start>"
# END = "<end>"
# UNK = "<unk>"
# PAD = "<pad>"
# BD = "<bd>"
# SPECIALS = [START,END,UNK,PAD, BD]

In [ ]:
langs_codes = {"Gitksan": "git", "Arapaho":"arp", "Lezgi":"lez", "Nyangbo":"nyb", "Tsez":"ddo","Uspanteko":"usp"}

def tag_bd(seq):
    for i, c in enumerate(seq):
        if c == " ":
            seq[i] = "<bd>"
    return seq

# gloss_pipeline = lambda gloss: tok(tag_gloss(gloss.split("-")))
# morph_pipeline = lambda morph: list(morph)
# word_pipeline = lambda words: list(words.lower())


In [ ]:

def get_lang_df(lang):
    ld = {
        "train": defaultdict(list),
        "dev": defaultdict(list)
    }
        
    path = f"data/{lang}/"
    train_fn = f"{langs_codes[lang]}-train-track2-uncovered"
    dev_fn = f"{langs_codes[lang]}-dev-track2-uncovered"

    for fp in [train_fn, dev_fn]:
        data_type = fp.split("-")[1]
        with open(path + fp, "r") as f:
            for line in f.readlines():
                

                if line.startswith("\\t"):
                    ld[data_type]["transcription"].append(normalize(line.lstrip("\\t ").rstrip("\n")))
                if line.startswith("\g"):
                    ld[data_type]["gloss"].append(normalize(line.lstrip("\\g ").rstrip("\n")))
                if line.startswith("\m"):
                    ld[data_type]["morphemes"].append(normalize(line.lstrip("\\m ").rstrip("\n")))

                if line.startswith("\l"):
                    ld[data_type]["translation"].append(normalize(line.lstrip("\\l ").rstrip("\n")))
    
    train_df = pd.DataFrame.from_dict(ld["train"])
    dev_df = pd.DataFrame.from_dict(ld["dev"])
    
    return train_df,dev_df



In [ ]:

def yield_input(data):
    #data = data.loc[:, "input"].apply(lambda x: list(x)) + data.loc[:, "msd"].apply(lambda x: x.split(";")).apply(lambda x: ["FEAT="+ l for l in x])
    for ex in data:#.iloc[:, 0].apply(lambda x: list(x)) + data.iloc[:, 2].apply(lambda x: x.split(";")).apply(lambda x: ["FEAT="+ l for l in x]):
        

        #if ex == "msd":
            #data[ex].apply(lambda x: x.split(";")).apply(lambda x: ["FEAT="+ l for l in x])
        yield ["<start>"] +tag_bd(list(ex.transcription)) + ["<end>"]

    
def yield_output(data):
    for ex in data:#.iloc[:, 1].apply(lambda x:list(x)):
        yield ["<start>"] + list(ex.morphemes) + ["<end>"]

In [ ]:

class UDDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        self.iloc = data.iloc
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.iloc[index]   
    

In [ ]:

def get_vocab(lang,batch_size):

    train_df, dev_df = get_lang_df(lang)
    
    train = UDDataset(train_df)
    dev = UDDataset(dev_df)
    
    vocab = build_vocab_from_iterator(chain(yield_input(train), yield_output(train)), specials=SPECIALS)
    vocab.set_default_index(vocab[UNK]) 
    
    sequence_transform = lambda s:[vocab[c] for c in s]
    
    def collate_batch(batch):
            input, output = [], []
            for orth, morph in zip(yield_input(batch), yield_output(batch)):
                input_tensor = torch.tensor(sequence_transform(orth), dtype=torch.long)
                output_tensor = torch.tensor(sequence_transform(morph), dtype=torch.long)
                input.append(input_tensor)
                output.append(output_tensor)
            
            return pad_sequence(input, batch_first=False, padding_value=vocab[PAD]), pad_sequence(output, batch_first=False, padding_value=vocab[PAD])
        
    train_iter = DataLoader(train, batch_size=1, shuffle=True, collate_fn=collate_batch)
    dev_iter = DataLoader(dev, batch_size=1, shuffle=True, collate_fn=collate_batch)
    
    return train,dev_iter,vocab

        

    
    
    
    
        